In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

In [2]:
player_ids = pd.read_csv('files/SFBB Player ID Map - PLAYERIDMAP.csv')
player_ids = player_ids[['MLBNAME', 'MLBID']].copy()
display([(i, player_ids[i].dtype) for i in player_ids.columns])
player_ids = player_ids.dropna(subset=['MLBID']).copy()
player_ids['MLBID'] = player_ids['MLBID'].astype('int64')
display([(i, player_ids[i].dtype) for i in player_ids.columns])
catcher_names = player_ids.drop_duplicates(subset='MLBID').set_index('MLBID')['MLBNAME']

[('MLBNAME', dtype('O')), ('MLBID', dtype('float64'))]

[('MLBNAME', dtype('O')), ('MLBID', dtype('int64'))]

In [3]:
savant_2024 = pd.read_csv('files/2024/savantdata-2024.csv')

In [4]:
savant_2024['catcher_name'] = savant_2024['fielder_2'].map(catcher_names)
print("Missing names:", savant_2024['catcher_name'].isna().sum())
print("Total rows:", len(savant_2024))

Missing names: 16421
Total rows: 708825


In [5]:
display(savant_2024.shape)
display(savant_2024.head())

(708825, 114)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,catcher_name
0,FF,2024-09-30,97.4,-2.10,4.88,"Díaz, Edwin",518595,621242,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,X,6.0,ground_ball,2,2,2024,-0.96,0.99,0.67,3.00,NaN,642201.0,NaN,2,9,Bot,112.78,146.97,NaN,NaN,NaN,NaN,9.893410,-141.548522,-1.857108,-15.148286,30.423852,-18.544662,3.43,1.54,5.0,87.6,-30.0,99.9,2196.0,7.6,747139,682626,624413,657193,578428,596019,607043,621438,516782,52.91,0.049,0.052,0.0,1.0,0.0,0.0,2.0,82,5,4-Seam Fastball,7,8,7,8,8,7,7,8,Standard,Strategic,232.0,-0.142,-0.248,68.8,7.3,0.059,0.248,88.0,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,1.40,0.96,0.96,17.6,Francisco Alvarez
1,SL,2024-09-30,90.7,-2.14,5.06,"Díaz, Edwin",518595,621242,NaN,ball,NaN,NaN,NaN,NaN,14.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,B,NaN,NaN,1,2,2024,0.20,0.61,0.75,1.20,NaN,642201.0,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,6.952856,-131.970548,-5.159843,1.012613,25.103830,-24.224355,3.47,1.52,NaN,NaN,NaN,92.9,2209.0,7.3,747139,682626,624413,657193,578428,596019,607043,621438,516782,53.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,4,Slider,7,8,7,8,8,7,7,8,Standard,Strategic,201.0,0.000,0.037,NaN,NaN,NaN,-0.037,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.14,-0.20,-0.20,23.1,Francisco Alvarez
2,SL,2024-09-30,91.1,-2.07,5.14,"Díaz, Edwin",518595,621242,NaN,swinging_strike,NaN,NaN,NaN,NaN,9.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,S,NaN,NaN,1,1,2024,0.12,0.35,0.66,1.61,NaN,642201.0,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,6.757602,-132.670539,-3.857177,0.114032,25.108326,-27.443443,3.43,1.54,NaN,NaN,NaN,93.5,2302.0,7.4,747139,682626,624413,657193,578428,596019,607043,621438,516782,53.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,3,Slider,7,8,7,8,8,7,7,8,Standard,Strategic,210.0,0.000,-0.060,71.2,8.9,NaN,0.060,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.37,-0.12,-0.12,22.4,Francisco Alvarez
3,SL,2024-09-30,91.3,-2.05,5.07,"Díaz, Edwin",518595,621242,NaN,ball,NaN,NaN,NaN,NaN,14.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,B,NaN,NaN,0,1,2024,0.21,0.63,0.61,1.18,NaN,642201.0,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,6.385514,-132.826939,-5.397765,1.181254,26.329685,-23.817729,3.47,1.54,NaN,NaN,NaN,93.5,2227.0,7.4,747139,682626,624413,657193,578428,596019,607043,621438,516782,53.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,2,Slider,7,8,7,8,8,7,7,8,Standard,Strategic,212.0,0.000,0.012,NaN,NaN,NaN,-0.012,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.09,-0.21,-0.21,22.4,Francisco Alvarez
4,SL,2024-

In [6]:
display(savant_2024.columns[savant_2024.isna().sum() / len(savant_2024) >= 0.9])
old_system_cols = ['spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire']

savant_2024 = savant_2024.drop(columns = old_system_cols).copy()
savant_2024 = savant_2024.dropna(axis=0, subset=['pitch_type', 'zone']).copy()

Index(['spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated',
       'break_length_deprecated', 'on_3b', 'tfs_deprecated',
       'tfs_zulu_deprecated', 'umpire', 'sv_id'],
      dtype='object')

In [7]:
savant_2024.loc[:, 'catcher_name'] = savant_2024['fielder_2'].map(catcher_names).astype('object')
savant_2024['catcher_name']

0         Francisco Alvarez
1         Francisco Alvarez
2         Francisco Alvarez
3         Francisco Alvarez
4         Francisco Alvarez
                ...        
708820        Logan O'Hoppe
708821        Logan O'Hoppe
708822        Logan O'Hoppe
708823        Logan O'Hoppe
708824        Logan O'Hoppe
Name: catcher_name, Length: 708540, dtype: object

In [8]:
# Total rows in savant_2024 before anything
print("Total rows:", len(savant_2024))

# Unique fielder_2 values in savant_2024
print("Unique fielder_2 values:", savant_2024['fielder_2'].nunique())

# How many of those values are in the catcher_names index?
print("Matched fielder_2 IDs:", savant_2024['fielder_2'].isin(catcher_names.index).sum())

# How many fielder_2 IDs don't exist in catcher_names (i.e., won't map)
print("Unmatched fielder_2 IDs:", (~savant_2024['fielder_2'].isin(catcher_names.index)).sum())

Total rows: 708540
Unique fielder_2 values: 100
Matched fielder_2 IDs: 692119
Unmatched fielder_2 IDs: 16421


In [9]:
savant_2024.loc[:, 'catcher_name'] = savant_2024['catcher_name'].fillna('Unknown Catcher')

In [10]:
shadow_zone_pitches = savant_2024[savant_2024['zone'].isin([11.0, 12.0, 13.0, 14.0])].copy()


In [15]:
useful_catcher_cols = ['catcher_name', 'player_name', 'events', 
                       'description', 'balls', 'strikes', 'zone',
                       'plate_x', 'plate_z', 'sz_top', 'sz_bot', 
                       'woba_value', 'woba_denom','delta_pitcher_run_exp', 
                       'delta_home_win_exp']


catcher_data = shadow_zone_pitches[useful_catcher_cols].reset_index(drop=True).copy()

In [22]:
strike_cols = ['swinging_strike', 'foul', 'hit_into_play', 'swinging_strike_blocked', 'called_strike', 'foul_tip', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip']

catcher_data.loc[:, 'is_strike'] = (catcher_data['description'].isin(strike_cols)).astype(int).copy()
catcher_data.loc[:, 'is_ball'] = (~catcher_data['description'].isin(strike_cols)).astype(int).copy()

In [23]:
catcher_data

,catcher_name,player_name,events,description,balls,strikes,zone,plate_x,plate_z,sz_top,sz_bot,woba_value,woba_denom,delta_pitcher_run_exp,delta_home_win_exp,is_strike,is_ball
0,Francisco Alvarez,"Díaz, Edwin",NaN,ball,1,2,14.0,0.75,1.20,3.47,1.52,NaN,NaN,-0.037,0.000,0,1
1,Francisco Alvarez,"Díaz, Edwin",NaN,ball,0,1,14.0,0.61,1.18,3.47,1.54,NaN,NaN,-0.012,0.000,0,1
2,Francisco Alvarez,"Díaz, Edwin",NaN,swinging_strike,0,0,14.0,1.36,1.78,3.43,1.54,NaN,NaN,0.040,0.000,1,0
3,Francisco Alvarez,"Díaz, Edwin",strikeout,swinging_strike,1,2,14.0,1.25,2.22,3.33,1.53,0.0,1.0,0.198,-0.136,1,0
4,Francisco Alvarez,"Díaz, Edwin",NaN,blocked_ball,0,2,14.0,1.66,0.51,3.31,1.55,NaN,NaN,-0.024,0.000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356955,Jacob Stallings,"Feltner, Ryan",NaN,ball,1,1,14.0,1.84,0.63,3.44,1.51,NaN,NaN,-0.058,0.000,0,1
356956,Jacob Stallings,"Feltner, Ryan",NaN,ball,0,0,11.0,-0.86,3.86,3.30,1.49,NaN,NaN,-0.040,0.000,0,1
356957,Jacob Stallings,"Feltner, Ryan",NaN,foul,2,2,12.0,0.84,3.36,3.28,1.50,NaN,NaN,0.000,0.000,1,0
356958,Jacob Stallings,"Feltner, Ryan",NaN,ball,1,2,14.0,1.58,0.66,3.48,1.51,NaN,NaN,-0.043,0.000,0,1


In [17]:
catcher_data['description'].unique()

array(['ball', 'swinging_strike', 'blocked_ball', 'foul', 'hit_into_play',
       'swinging_strike_blocked', 'hit_by_pitch', 'called_strike',
       'foul_tip', 'foul_bunt', 'pitchout', 'missed_bunt',
       'bunt_foul_tip'], dtype=object)

In [18]:
catcher_data['events'].unique()

array([nan, 'strikeout', 'field_out', 'walk', 'single', 'hit_by_pitch',
       'double', 'force_out', 'grounded_into_double_play', 'truncated_pa',
       'triple', 'fielders_choice_out', 'field_error', 'home_run',
       'double_play', 'sac_fly', 'catcher_interf', 'fielders_choice',
       'sac_bunt', 'strikeout_double_play', 'sac_fly_double_play',
       'triple_play'], dtype=object)

In [13]:
delta_run_exp_col = savant_2024.columns.get_loc('delta_run_exp')
new_statcast = savant_2024.iloc[:, delta_run_exp_col+1:].copy()
new_statcast

,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,catcher_name
0,68.8,7.3,0.059,0.248,88.0,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,1.40,0.96,0.96,17.6,Francisco Alvarez
1,NaN,NaN,NaN,-0.037,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.14,-0.20,-0.20,23.1,Francisco Alvarez
2,71.2,8.9,NaN,0.060,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.37,-0.12,-0.12,22.4,Francisco Alvarez
3,NaN,NaN,NaN,-0.012,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.09,-0.21,-0.21,22.4,Francisco Alvarez
4,71.7,9.0,NaN,0.040,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.20,-0.17,-0.17,20.2,Francisco Alvarez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708820,NaN,NaN,NaN,0.041,NaN,0,0,0.537,0.463,30,26,31,26,1,0,6.0,1.0,6.0,1.0,1.16,0.63,0.63,36.7,Logan O'Hoppe
708821,78.5,7.4,0.064,0.243,88.0,0,0,0.522,0.478,30,22,31,23,1,0,6.0,1.0,6.0,1.0,2.00,1.53,1.53,39.9,Logan O'Hoppe
708822,NaN,NaN,NaN,-0.024,NaN,0,0,0.522,0.478,30,22,31,23,1,0,6.0,1.0,6.0,1.0,3.10,-0.37,-0.37,37.0,Logan O'Hoppe
708823,NaN,NaN,NaN,0.037,NaN,0,0,0.522,0.478,30,22,31,23,1,0,6.0,1.0,6.0,1.0,2.02,1.30,1.30,34.7,Logan O'Hoppe
